In [1]:
import matplotlib.pyplot as plt

from keras.models import Sequential, load_model
from keras.layers import Dense, Convolution2D, Flatten
from keras.optimizers import RMSprop
import tensorflow as tf

import gym
import numpy as np
import math
import cv2
import random
from collections import deque, namedtuple

from wrappers import wrap_deepmind

Using TensorFlow backend.


In [2]:
env = gym.make('BreakoutNoFrameskip-v4')
n_actions = env.action_space.n
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



class DeepQNet(nn.Module):
    def __init__(self, h, w):
        super(DeepQNet, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size= 8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)

        def conv2d_size_out(size, kernel_size, stride):
            return (size - (kernel_size - 1) - 1) // stride + 1
        
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w, 8, 4), 4, 2), 3, 1)
        convh = conv2d_size_out(conv2d_size_out(
            conv2d_size_out(h, 8, 4), 4, 2), 3, 1)

        linear_input = convh * convw * 64
        self.fc1 = nn.Linear(linear_input, 512)
        self.out = nn.Linear(512, n_actions)


    
    def forward(self, x):
        x = F.relu(self.conv1(x.float()))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        
        x = F.relu(self.fc1(x.view(x.size(0), -1)))
        x = self.out(x)
        return x

NameError: name 'nn' is not defined

In [ ]:
Experience = namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])



class ReplayMemory:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), np.array(next_states)

In [ ]:
def get_epsilon(current_step):
    rate = (EPS_END-EPS_START)/MEM_SIZE
    eps_threshold = rate * current_step + EPS_START
    if eps_threshold < EPS_END:
        return EPS_END
    return eps_threshold


def select_action(state, steps_done, eval=False):
    global EPSILON

    # This equation is for the decaying epsilon
    eps_threshold = get_epsilon(steps_done)

    if eval:
        eps_threshold = EPS_END

    r = np.random.rand()

    EPSILON = eps_threshold

    # We select an action with an espilon greedy policy 
    if r > eps_threshold:
        with torch.no_grad():
            # Return the action with the maximum Q value for the current state
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device)

In [ ]:
def optimize_model():
    if memory.current < memory.batch_size:
        return 0
    optimizer.zero_grad()
    states, actions, rewards, next_states, dones = memory.get_minibatch()
    states_v = torch.tensor(states).to(device)
    next_states_v = torch.tensor(next_states).to(device)
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done = torch.tensor(dones).to(device)

    state_action_values = policy_net(states_v).gather(
        1, actions_v.long().unsqueeze(-1)).squeeze(-1)
    next_state_values = target_net(next_states_v).max(1)[0]
    next_state_values[done] = 0.0
    next_state_values = next_state_values.detach()

    expected_state_action_values = next_state_values * GAMMA + rewards_v
     
    # Compute Huber loss
    loss = nn.MSELoss()(state_action_values, expected_state_action_values)
    # Optimize the model
    loss.backward()
    optimizer.step() 
    
    return loss.item()